---
# **LAB 3 - Modello di esecuzione CUDA**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## [GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
#@title Bash setup
%%writefile /root/.bashrc

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# don't put duplicate lines in the history. See bash(1) for more options
# ... or force ignoredups and ignorespace
HISTCONTROL=ignoredups:ignorespace

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=10000
HISTFILESIZE=20000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

PS1='\[\033[01;34m\]\w\[\033[00m\]\$ '

# enable color support of ls and also add handy aliases
if [ -x /usr/bin/dircolors ]; then
    test -r ~/.dircolors && eval "$(dircolors -b ~/.dircolors)" || eval "$(dircolors -b)"
    alias ls='ls --color=auto'
    #alias dir='dir --color=auto'
    #alias vdir='vdir --color=auto'

    alias grep='grep --color=auto'
    alias fgrep='fgrep --color=auto'
    alias egrep='egrep --color=auto'
fi

# some more ls aliases
alias ll='ls -lF'
alias la='ls -A'
alias l='ls -CF'

# path setup
export PATH="./:/usr/local/cuda/bin:$PATH"

In [ ]:
!source /root/.bashrc

In [ ]:
# path setup
!mkdir -p /content/utils
!mkdir -p divergence
!mkdir -p histogram
!mkdir -p MQDB-CUDA
!mkdir -p parReduce

# ▶️ VS Code on Colab

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "gpu" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="https://github.com/giulianogrossi/GPUcomputing" #@param {type: "string"}
init_git_cloudflared(repository_url)

# ✅ Divergence analysis

In [ ]:
%%writefile /content/utils/common.h
#include <sys/time.h>

#ifndef _COMMON_H
#define _COMMON_H

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
    }                                                                          \
}

#define CHECK_CUBLAS(call)                                                     \
{                                                                              \
    cublasStatus_t err;                                                        \
    if ((err = (call)) != CUBLAS_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CUBLAS error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CURAND(call)                                                     \
{                                                                              \
    curandStatus_t err;                                                        \
    if ((err = (call)) != CURAND_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CURAND error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUFFT(call)                                                      \
{                                                                              \
    cufftResult err;                                                           \
    if ( (err = (call)) != CUFFT_SUCCESS)                                      \
    {                                                                          \
        fprintf(stderr, "Got CUFFT error %d at %s:%d\n", err, __FILE__,        \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUSPARSE(call)                                                   \
{                                                                              \
    cusparseStatus_t err;                                                      \
    if ((err = (call)) != CUSPARSE_STATUS_SUCCESS)                             \
    {                                                                          \
        fprintf(stderr, "Got error %d at %s:%d\n", err, __FILE__, __LINE__);   \
        cudaError_t cuda_err = cudaGetLastError();                             \
        if (cuda_err != cudaSuccess)                                           \
        {                                                                      \
            fprintf(stderr, "  CUDA error \"%s\" also detected\n",             \
                    cudaGetErrorString(cuda_err));                             \
        }                                                                      \
        exit(1);                                                               \
    }                                                                          \
}

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

inline void device_name() {
    // set up device
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));
}

typedef unsigned long ulong;
typedef unsigned int uint;

#endif // _COMMON_H

In [ ]:
%%writefile /content/divergence/div.cu
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "../utils/common.h"

/*
 * Kernel with warp divergence
 */
__global__ void evenOddDIV(int *c, const ulong N) {
	ulong tid = blockIdx.x * blockDim.x + threadIdx.x;
	int a, b;

	if (!(tid % 2))   // branch divergence
		a = 2;                  
	else
		b = 1;                  

	// check index
	if (tid < N)
		c[tid] = a + b;
}

/*
 * Kernel without warp divergence
 */
__global__ void evenOddNODIV(int *c, const int N) {
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	int a = 0, b = 0;
	unsigned int i, twoWarpSize = 2 * warpSize;

	int wid = tid / warpSize; 	// warp index wid = 0,1,2,3,...
	if (!(wid % 2))
		a = 2;                  // branch1: thread tid = 0-31, 64-95, ...
	else
		b = 1;                  // branch2: thread tid = 32-63, 96-127, ...

	// right index
	if (!(wid % 2))  // even
		i = 2 * (tid % warpSize) + (tid / twoWarpSize) * twoWarpSize;
	else            // odd
		i = 2 * (tid % warpSize) + 1 + (tid / twoWarpSize) * twoWarpSize;

	// check index
	if (i < N) {
		c[i] = a + b;
	}
}

/*
 * MAIN
 */
int main(int argc, char **argv) {

	// set up data size
	int blocksize = 1024;
	ulong size = 1024*1024;

	if (argc > 1)
		blocksize = atoi(argv[1]);
	if (argc > 2)
		size = atoi(argv[2]);
	ulong nBytes = size * sizeof(int);

	printf("Data size: %lu  -- ", size);
  printf("Data size (bytes): %lu MB\n", nBytes/1000000);

	// set up execution configuration
	dim3 block(blocksize, 1);
	dim3 grid((size + block.x - 1) / block.x, 1);
	printf("Execution conf (block %d, grid %d)\nKernels:\n", block.x, grid.x);

	// allocate memory
	int *d_C, *C;
	C = (int *) malloc(nBytes);
	CHECK(cudaMalloc((void** )&d_C, nBytes));

	// run kernel 1
	double iStart, iElaps;
	iStart = seconds();
	evenOddDIV<<<grid, block>>>(d_C, size);
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	printf("\tevenOddDIV<<<%d, %d>>> elapsed time %f sec \n\n", grid.x, block.x, iElaps);
	CHECK(cudaGetLastError());
  
  CHECK(cudaMemcpy(C, d_C, nBytes, cudaMemcpyDeviceToHost));


	// run kernel 2
  CHECK(cudaMemset(d_C, 0.0, nBytes)); // reset memory
	iStart = seconds();
	evenOddNODIV<<<grid, block>>>(d_C, size);
	iElaps = seconds() - iStart;
	printf("\tevenOddNODIV<<<%d, %d>>> elapsed time %f sec \n\n", grid.x, block.x, iElaps);
	CHECK(cudaGetLastError());

	CHECK(cudaMemcpy(C, d_C, nBytes, cudaMemcpyDeviceToHost));

	free(C);
	// free gpu memory and reset device
	CHECK(cudaFree(d_C));
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}


In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37 divergence/div.cu -o div 
!./div 1024 2000000000

In [ ]:
# Compilazione ed esecuzione versione di debug 
!nvcc -arch=sm_37 -g -G divergence/div.cu -o div_deb
!./div_deb 1024 2000000000

# ✅ Parallel Reduction

In [ ]:
%%writefile /content/parReduce/preduce.cu
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#include "../utils/common.h"

/*
 *  Block by block parallel implementation with divergence (sequential schema)
 */
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
 *  Block by block parallel implementation without divergence (interleaved schema)
 */
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)  {
		if (tid < stride)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}


/*
 * MAIN: test on parallel reduction
 */
int main(void) {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;            // block dim 1D
	ulong numBlock = 1024*1024;      // grid dim 1D
	ulong n = blockSize * numBlock;  // array dim
	long sum_CPU = 0, sum_GPU;
	long nByte = n*sizeof(int), mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	for (ulong i = 0; i < n; i++) a[i] = 1;

	CHECK(cudaMalloc((void **) &d_a, nByte));
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void **) &d_b, mByte));
	CHECK(cudaMemset((void *) d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++) 
    sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	
  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	
  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	// reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i]=1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());
	
  // memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));
	
  // check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
  //		printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);
	
  // reset input vector on GPU
	for (ulong i = 0; i < n; i++) a[i] = 1;
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++)
		sum_GPU += b[i];
	assert(sum_GPU == n);

	cudaFree(d_a);

	CHECK(cudaDeviceReset());
	return 0;
}


In [ ]:
#Compilazione ed esecuzione

!nvcc -arch=sm_37 parReduce/preduce.cu -o preduce
!./preduce

# 🔴 TODO

Analizzare le prestazioni usando 
* `nvprof --events branch,divergent_branch`
* `nvprof --metrics achieved_occupancy`

# ✅ Istogramma di un'immagine BMP

In [ ]:
%%writefile /content/utils/bmpUtil.h
struct imgBMP {
	int width;
	int height;
	unsigned char headInfo[54];
	unsigned long int rowByte;
} img;

#define	WIDTHB		img.rowByte
#define	WIDTH		img.width
#define	HEIGHT		img.height
#define	IMAGESIZE	(WIDTHB*HEIGHT)

struct pixel {
	unsigned char R;
	unsigned char G;
	unsigned char B;
};

typedef unsigned long ulong;
typedef unsigned int uint;
typedef unsigned char pel;    // pixel element

pel *ReadBMPlin(char*);         // Load a BMP image
void WriteBMPlin(pel *, char*); // Store a BMP image


In [ ]:
%%writefile /content/utils/ImageStuff.h

struct ImgProp {
	int Hpixels;
	int Vpixels;
	unsigned char HeaderInfo[54];
	unsigned long int Hbytes;
};

struct Pixel {
	unsigned char R;
	unsigned char G;
	unsigned char B;
};

typedef unsigned char pel;    // pixel element

pel** ReadBMP(char*);         // Load a BMP image
void WriteBMP(pel**, char*);  // Store a BMP image

extern struct ImgProp ip;

In [ ]:
%%writefile /content/utils/ImageStuff.c

#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#include "ImageStuff.h"

/*
 * Load a BMP image
 */

pel** ReadBMP(char* filename) {
	FILE* f = fopen(filename, "rb");
	if (f == NULL) {
		printf("\n\n%s NOT FOUND\n\n", filename);
		exit(1);
	}

	pel HeaderInfo[54];
	fread(HeaderInfo, sizeof(pel), 54, f); // read the 54-byte header

	// extract image height and width from header
	int width = *(int*) &HeaderInfo[18];
	int height = *(int*) &HeaderInfo[22];

	//copy header for re-use
	for (unsigned int i = 0; i < 54; i++)
		ip.HeaderInfo[i] = HeaderInfo[i];

	ip.Vpixels = height;
	ip.Hpixels = width;
	int RowBytes = (width * 3 + 3) & (~3);
	ip.Hbytes = RowBytes;

	printf("\n   Input BMP File name: %20s  (%u x %u)", filename, ip.Hpixels, ip.Vpixels);

	pel tmp;
	pel **TheImage = (pel **) malloc(height * sizeof(pel*));
	for (unsigned int i = 0; i < height; i++)
		TheImage[i] = (pel *) malloc(RowBytes * sizeof(pel));

	for (unsigned int i = 0; i < height; i++)
		fread(TheImage[i], sizeof(unsigned char), RowBytes, f);

	fclose(f);
	return TheImage;  // remember to free() it in caller!
}

/*
 * Store a BMP image
 */
void WriteBMP(pel** img, char* filename) {
	FILE* f = fopen(filename, "wb");
	if (f == NULL) {
		printf("\n\nFILE CREATION ERROR: %s\n\n", filename);
		exit(1);
	}

	//write header
	for (unsigned int x = 0; x < 54; x++)
		fputc(ip.HeaderInfo[x], f);

	//write data
	for (unsigned int x = 0; x < ip.Vpixels; x++)
		for (unsigned int y = 0; y < ip.Hbytes; y++) {
			char temp = img[x][y];
			fputc(temp, f);
		}

	printf("\n  Output BMP File name: %20s  (%u x %u)", filename, ip.Hpixels,
			ip.Vpixels);

	fclose(f);
}

# 🔴 TODO

Calcolare l'istogramma di un aimmagine BMP con uso di `atomicAdd`

In [ ]:
%%writefile /content/histogram/hist.cu
/**
 * hist.cu
 */
#include <cuda_runtime.h>
#include <stdio.h>
#include <time.h>
#include <limits.h>

#include "../utils/ImageStuff.h"
#include "../utils/bmpUtil.h"
#include "../utils/common.h"

/*
 * Kernel 1D that computes histogram on GPU
 */
__global__ void histogramBMP(...) {

	// TODO
	
}

/*
 * Function that computes histogram on CPU
 */
void hist_CPU(uint *bins, const pel *imgSrc, const uint W, const uint H, const uint M) {
	for (int i = 0; i < W*H; i++) {
		uint r = i / W;              // row of the source pixel
		uint off = i - r * W;        // col of the source pixel

		//  ** byte granularity **
		uint p = M * r + 3*off;      // src byte position of the pixel
		pel R = imgSrc[p];
		pel G = imgSrc[p+1];
		pel B = imgSrc[p+2];
		bins[R] += 1;
		bins[G+256] += 1;
		bins[B+512] += 1;
	}
}

int main(int argc, char **argv) {

	uint dimBlock = 1024;
	pel *imgBMP_CPU;     // Where images are stored in CPU
	pel *imgBMP_GPU;	 // Where images are stored in GPU

	uint *binsRGB_CPU, *binsRGB_GPU, *binsRGB_GPU2CPU;
	uint N_bins = 3*256;
	uint bin_size = N_bins*sizeof(uint);

	if (argc > 2)
		dimBlock = atoi(argv[2]);
	else if (argc < 2) {
		printf("\n\nUsage:  hist InputFilename dimBlock\n");
		exit(EXIT_FAILURE);
	}

	// bins for CPU & GPU
	binsRGB_CPU = (uint*) calloc(N_bins, sizeof(uint));
	binsRGB_GPU2CPU = (uint*) malloc(bin_size);
	CHECK(cudaMalloc((void**) &binsRGB_GPU, bin_size));

	// Create CPU memory to store the input image
	imgBMP_CPU = ReadBMPlin(argv[1]);
	if (imgBMP_CPU == NULL) {
		printf("Cannot allocate memory for the input image...\n");
		exit(EXIT_FAILURE);
	}

	// Allocate GPU buffer for image and bins
	CHECK(cudaMalloc((void**) &imgBMP_GPU, IMAGESIZE));

	// Copy input vectors from host memory to GPU buffers.
	CHECK(cudaMemcpy(imgBMP_GPU, imgBMP_CPU, IMAGESIZE, cudaMemcpyHostToDevice));

	// CPU histogram
	double start = seconds();   // start time
	hist_CPU(binsRGB_CPU, imgBMP_CPU, WIDTH, HEIGHT, WIDTHB);
	double stop = seconds();   // elapsed time
	printf("\nCPU elapsed time %f sec \n\n", stop - start);

	// invoke kernels (define grid and block sizes)
	uint nPixels = WIDTH*HEIGHT;
	int dimGrid = (nPixels + dimBlock - 1) / dimBlock;
	printf("\ndimGrid = %d   dimBlock = %d\n",dimGrid,dimBlock);

	start = seconds();   // start time

	// TODO histogramBMP<<<...>>>(...);
	
	CHECK(cudaDeviceSynchronize());
	stop = seconds();   // elapsed time
	printf("\nGPU elapsed time %f sec \n\n", stop - start);

	// Copy output (results) from GPU buffer to host (CPU) memory.
	CHECK(cudaMemcpy(binsRGB_GPU2CPU, binsRGB_GPU, bin_size, cudaMemcpyDeviceToHost));

	for (int i = 0; i < N_bins/3; i++)
		printf("bin_GPU[%d] = \t%d\t%d\t%d\t -- bin_CPU[%d] = \t%d\t%d\t%d\n", i,
				binsRGB_GPU2CPU[i],binsRGB_GPU2CPU[i+256],binsRGB_GPU2CPU[i+512],
				i,binsRGB_CPU[i],binsRGB_CPU[i+256],binsRGB_CPU[i+512]);

	// Deallocate GPU memory
	cudaFree(imgBMP_GPU);
	cudaFree(binsRGB_GPU);

	// tracing tools spel as Parallel Nsight and Visual Profiler to show complete traces.
	CHECK(cudaDeviceReset());

	return (EXIT_SUCCESS);
}

/*
 *  Read a 24-bit/pixel BMP file into a 1D linear array.
 *  Allocate memory to store the 1D image and return its pointer
 */
pel *ReadBMPlin(char* fn) {
	static pel *Img;
	FILE* f = fopen(fn, "rb");
	if (f == NULL) {
		printf("\n\n%s NOT FOUND\n\n", fn);
		exit(EXIT_FAILURE);
	}

	pel HeaderInfo[54];
	size_t nByte = fread(HeaderInfo, sizeof(pel), 54, f); // read the 54-byte header
	// extract image height and width from header
	int width = *(int*) &HeaderInfo[18];
	img.width = width;
	int height = *(int*) &HeaderInfo[22];
	img.height = height;
	int RowBytes = (width * 3 + 3) & (~3);  // row is multiple of 4 pixel
	img.rowByte = RowBytes;
	//save header for re-use
	memcpy(img.headInfo, HeaderInfo, 54);
	printf("\n Input File name: %5s  (%d x %d)   File Size=%lu", fn, img.width, img.height, IMAGESIZE);

	// allocate memory to store the main image (1 Dimensional array)
	Img = (pel *) malloc(IMAGESIZE);
	if (Img == NULL)
		return Img;      // Cannot allocate memory
	// read the image from disk
	size_t out = fread(Img, sizeof(pel), IMAGESIZE, f);
	fclose(f);
	return Img;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_37 histogram/hist.cu utils/ImageStuff.c -o hist
!hist

# ✅ Prodotto MQDB CUDA

In [ ]:
%%writefile /content/MQDB-CUDA/mqdb.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#ifndef MQDB_H
#define MQDB_H

#define randu() ((float)rand() / (float) RAND_MAX)
#define abs(x) ((x)<0 ? (-x) : (x))

typedef unsigned long ulong;
typedef unsigned int uint;

typedef struct MQDB {
	char desc[100];   // description
	int nBlocks;      // num. of blocks
	int *blkSize;     // block dimensions
	float *elem;       // elements in row-major order
	ulong nElems;     // actual number of elements
} mqdb;

// function prototypes
int genRandDims(mqdb*, uint, uint, int);
int genRandDimsUnified(mqdb*, uint, uint, int);
void fillBlocks(mqdb*, uint, uint, char, float);
void fillBlocksUnified(mqdb*, uint, uint, char, float);
mqdb mqdbConst(uint, uint, uint, float);
void mqdbProd(mqdb, mqdb, mqdb);
void matProd(mqdb, mqdb, mqdb);
void checkResult(mqdb, mqdb);
void mqdbDisplay(mqdb*);

#endif


In [ ]:
%%writefile /content/MQDB-CUDA/mqdb.cpp
#include "mqdb.h"

/**
 * random generate block dimensions
 */
int genRandDims(mqdb *M, uint n, uint k, int seed) {

	if (n == 0 || k == 0 || k > n) {
		printf("error: n and k must be positive and n > k!\n");
		return(-1);
	}
	srand(seed);
	M->nBlocks = k;
	// random generation of block sizes
	M->blkSize = (int *) malloc(k * sizeof(int));
	int sum = 0;
	int r;
	float mu = 2.0f * (float) n / (float) k;
	for (int i = 0; i < k - 1; i++) {
		// expected value E[block_size] = n/k
		while ((r = round(mu * randu())) > n - sum - k + i + 1);
		if (!r)
			r += 1;
		M->blkSize[i] = r;
		sum += r;
	}
	M->blkSize[k - 1] = n - sum;
	return(0);
}

/**
 * fill blocks either random or constant
 */
void fillBlocks(mqdb *M, uint n, uint k, char T, float c) {
	//mat size n*n
	M->elem = (float *) calloc(n * n, sizeof(float));
	M->nElems = 0;
	int offset = 0;
	// loop on blocks
	for (int i = 0; i < k; i++) {
		for (int j = 0; j < M->blkSize[i]; j++)
			for (int k = 0; k < M->blkSize[i]; k++) {
				if (T == 'C')  	    // const fill mat entries
					M->elem[offset * n + j * n + k + offset] = c;
				else if (T == 'R') 	// random fill mat entries
					M->elem[offset * n + j * n + k + offset] = c*randu();
				//printf("M->[%d] = %f\n", offset * n + j * n + k + offset, M->elem[offset * n + j * n + k + offset]);
	}
		offset += M->blkSize[i];
		M->nElems += M->blkSize[i]*M->blkSize[i];
	}
	// set description
	sprintf(M->desc, "Random mqdb:  mat. size = %d, num. blocks = %d",n,k);
}


/**
 * random generate block dimensions - using CUDA Unified Memory
 */
int genRandDimsUnified(mqdb *M, uint n, uint k, int seed) {

	if (n == 0 || k == 0 || k > n) {
		printf("error: n and k must be positive and n > k!\n");
		return(-1);
	}
	srand(seed);
	M->nBlocks = k;
	int sum = 0;
	int r;
	float mu = 2.0f * (float) n / (float) k;
	for (int i = 0; i < k - 1; i++) {
		// expected value E[block_size] = n/k
		while ((r = round(mu * randu())) > n - sum - k + i + 1);
		if (!r)
			r += 1;
		M->blkSize[i] = r;
		sum += r;
	}
	M->blkSize[k - 1] = n - sum;
	return(0);
}

/**
 * fill blocks either random or constant - using CUDA Unified Memory
 */
void fillBlocksUnified(mqdb *M, uint n, uint k, char T, float c) {
	//mat size n*n
	M->nElems = 0;
	int offset = 0;
	// loop on blocks
	for (int i = 0; i < k; i++) {
		for (int j = 0; j < M->blkSize[i]; j++)
			for (int k = 0; k < M->blkSize[i]; k++) {
				if (T == 'C')  	    // const fill mat entries
					M->elem[offset * n + j * n + k + offset] = c;
				else if (T == 'R') 	// random fill mat entries
					M->elem[offset * n + j * n + k + offset] = c*randu();
				//printf("M->[%d] = %f\n", offset * n + j * n + k + offset, M->elem[offset * n + j * n + k + offset]);
	}
		offset += M->blkSize[i];
		M->nElems += M->blkSize[i]*M->blkSize[i];
	}
	// set description
	sprintf(M->desc, "Random mqdb:  mat. size = %d, num. blocks = %d",n,k);
}

/**
 * rand_gen_mqdb: mqdb  type returned
 *                n     square matrix size
 *                k     number of blocks
 *                seed  seed for random generator
 */
mqdb genRandMat(unsigned n, unsigned k, unsigned seed) {
	mqdb M;
	genRandDims(&M, n, k, seed);

	// random fill mat entries
	fillBlocks(&M, n, k, 'R', 1.0);

	return M;
}

/**
 * const_mqdb: mqdb  is the type returned
 *                n     is the square matrix size
 *                k     is the number of blocks
 *                seed  is the seed for random generator
 *                c   	is the constant value assigned
 */
mqdb mqdbConst(uint n, uint k, uint seed, float c) {
	mqdb M;
	genRandDims(&M, n, k, seed);

	// fill mat entries with a constant
	fillBlocks(&M, n, k, 'C', c);

	return M;
}

/*
 * product between mqdb matrices restricted to blocks
 */
void mqdbProd(mqdb A, mqdb B, mqdb C) {
	uint n = 0;
	for (uint i = 0; i < A.nBlocks; i++)
		n += A.blkSize[i];                  // mat dim
	int k = A.nBlocks;                      // num blks
	int dl = 0;                             // blk left bound
	int dr = 0;                             // blk left bound
	for (uint i = 0; i < k; i++) {           // loop on blks
		dr += A.blkSize[i];                 // blk right bound
		for (uint r = dl; r < dr; r++) {     // scan block rows
			for (uint c = dl; c < dr; c++) { // scan block cols
				float s = 0;
				for (uint l = dl; l < dr; l++)
					s += A.elem[r*n + l] * B.elem[c + l * n];
				C.elem[r*n + c] = s;
			}
		}
		dl = dr;
	}
}

/*
 * standard (naive) matrix product on host
 */
void matProd(mqdb A, mqdb B, mqdb C) {
	int n = 0;
	for (uint i = 0; i < A.nBlocks; i++)
		n += A.blkSize[i];

	for (uint r = 0; r < n; r++)
		for (uint c = 0; c < n; c++) {
			double sum = 0;
			for (uint l = 0; l < n; l++){
				double a = A.elem[r * n + l];
				double b = B.elem[l * n + c];
				sum += a*b;
			}
			C.elem[r * n + c] = (float)sum;
		}
}

/*
 * elementwise comparison between two mqdb
 */
void checkResult(mqdb A, mqdb B) {
	double epsilon = 1.0E-8;
	bool match = 1;
	int n = 0;
	for (int i = 0; i < A.nBlocks; i++)
		n += A.blkSize[i];
	for (int i = 0; i < n * n; i++) {
		if (abs(A.elem[i] - B.elem[i]) > epsilon) {
			match = 0;
			printf("   * Arrays do not match!\n");
			printf("     gpu: %2.2f,  host: %2.2f at current %d\n", A.elem[i],
					B.elem[i], i);
			break;
		}
	}
	if (match)
		printf("   Arrays match\n\n");
}
/*
 * print mqdb
 */
void mqdbDisplay(mqdb *M) {
	int k = M->nBlocks;
	int n = 0;
	printf("%s\n", M->desc);
	printf("Block sizes [%d]: ", k);
	for (int j = 0; j < k; j++) {
		n += M->blkSize[j];
		printf("%d  ", M->blkSize[j]);
	}

	printf("\nElements: \n");
	for (int i = 0; i < n; i++) {
		for (int j = 0; j < n; j++) {
			if (M->elem[i*n + j] == 0)
				printf("------");
			else
				printf("%5.2f ", M->elem[i*n + j]);
		}
		printf("\n");
	}
	printf("\n");
}

# 🔴 TODO

Calcolare il prodotto di matrici MQDB con kernel CUDA 

In [ ]:
%%writefile /content/MQDB-CUDA/mqdb_prod.cu

#include "../MQDB-CUDA/mqdb.h"
#include "../utils/common.h"

#define BLOCK_SIZE 16     // block size

struct tms {
	double CPUtms;
	double GPUtmsNaive;
	double GPUtmsMQDB;
	float density;
};

/*
 * Kernel for standard (naive) matrix product
 */
__global__ void matProd(mqdb A, mqdb B, mqdb C, int n) {
	// row & col indexes
	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	// each thread computes an entry of the product matrix
	if ((row < n) && (col < n)) {
		float val = 0;
		for (int k = 0; k < n; k++)
			val += A.elem[row * n + k] * B.elem[k * n + col];
		C.elem[row * n + col] = val;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb
 */
__global__ void mqdbBlockProd(...) {
	
	// TODO
}

/*
 * Test on MQDB kernels
 */
void testKernelsMQDB(uint n, uint k, struct tms* times) {

	// mqdb host matrices
	mqdb A, B, C, C1;

	// mqdb device matrices
	mqdb d_A, d_B, d_C;

	// fill in
	A = mqdbConst(n, k, 10, 1);
	B = mqdbConst(n, k, 10, 1);
	C = mqdbConst(n, k, 10, 1);
	C1 = mqdbConst(n, k, 10, 1);

	ulong nBytes = n * n * sizeof(float);
	ulong kBytes = k * sizeof(uint);
	printf("Memory size required = %.1f (MB)\n",(float)nBytes/(1024.0*1024.0));

	// malloc and copy on device memory
	d_A.nBlocks = A.nBlocks;
	CHECK(cudaMalloc((void**)&d_A.blkSize, kBytes));
	CHECK(cudaMemcpy(d_A.blkSize, A.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_A.elem, nBytes));
	CHECK(cudaMemcpy(d_A.elem, A.elem, nBytes, cudaMemcpyHostToDevice));
	d_B.nBlocks = B.nBlocks;
	CHECK(cudaMalloc((void**)&d_B.blkSize, kBytes));
	CHECK(cudaMemcpy(d_B.blkSize, B.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_B.elem, nBytes));
	CHECK(cudaMemcpy(d_B.elem, B.elem, nBytes, cudaMemcpyHostToDevice));
	d_C.nBlocks = C.nBlocks;
	CHECK(cudaMalloc((void**)&d_C.blkSize, kBytes));
	CHECK(cudaMemcpy(d_C.blkSize, C.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_C.elem, nBytes));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*                    CPU MQDB product                     */
	/***********************************************************/
	printf("CPU MQDB product...\n");
	double start = seconds();
	mqdbProd(A,B,C);
	double CPUTime = seconds() - start;
	printf("   CPU elapsed time: %.5f (sec)\n\n", CPUTime);

	/***********************************************************/
	/*                     GPU mat product                     */
	/***********************************************************/
	printf("Kernel (naive) mat product...\n");
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	start = seconds();
	matProd<<<grid, block>>>(d_A, d_B, d_C, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime1 = seconds() - start;
	printf("   elapsed time:                %.2f (sec)\n", GPUtime1);
	printf("   speedup vs CPU MQDB product: %.2f\n", CPUTime/GPUtime1);
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));
	checkResult(C,C1);
	//	mqdbDisplay(C1);

	/***********************************************************/
	/*                     GPU MQDB product                    */
	/***********************************************************/
	printf("Kernel MQDB product...\n");
	 
	// TODO

	CHECK(cudaDeviceSynchronize());
	double GPUtime2 = seconds() - start;
	printf("   elapsed time:                    %.2f (sec)\n", GPUtime2);
	printf("   speedup vs CPU MQDB product:     %.2f\n", CPUTime/GPUtime2);
	printf("   speedup vs GPU std mat product:  %.2f\n", GPUtime1/GPUtime2);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));
	checkResult(C,C1);

	CHECK(cudaFree(d_A.elem));
	CHECK(cudaFree(d_B.elem));
	CHECK(cudaFree(d_C.elem));

	// collect times
	times->CPUtms = CPUTime;
	times->GPUtmsNaive = GPUtime1;
	times->GPUtmsMQDB = GPUtime2;
	
	float den = 0;
	for (uint j = 0; j < k; j++)
		den += A.blkSize[j]*A.blkSize[j];
	times->density = den/(n*n);
}

/*
 * main function
 */
int main(int argc, char *argv[]) {
	uint n = 2*1024;      // matrix size
	uint min_k = 30;       // max num of blocks
	uint max_k = 30;       // max num of blocks

	struct tms times[max_k-min_k+1];

	// multiple tests on kernels
	for (uint k = min_k; k <= max_k; k++) {
		printf("\n*****   k = %d --- (avg block size = %f)\n",k,(float)n/k);
		testKernelsMQDB(n, k, &times[k-min_k]);
	}

	FILE *fd;
	fd = fopen("res.csv", "w");
	if (fd == NULL) {
		perror("file error!\n");
		exit(1);
	}

	// write results on file
	fprintf(fd,"num blocks,");
		for (uint j = 0; j <= max_k-min_k; j++)
			fprintf(fd,"%d,",j+min_k);

	fprintf(fd,"\nCPU MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].CPUtms);

	fprintf(fd,"\nKernel mat product naive,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsNaive);

	fprintf(fd,"\nKernel MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDB);

	fprintf(fd,"\ndensity,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].density);

	fclose(fd);

	return 0;
}



In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_37 MQDB-CUDA/mqdb_prod.cu MQDB-CUDA/mqdb.cpp  -o mqdb_prod
!./mqdb_prod